In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
# import kagglehub
# moneshsoni_customer_review_path = kagglehub.dataset_download('moneshsoni/customer-review')
moneshsoni_customer_review_path = "./data/moneshsoni/customer-review"

print('Data source import complete.')


# Types of Chains in LangChain

LangChain provides several types of chains, each designed for a specific purpose.  
A **chain** is a structure that connects a **prompt (instructions)** with a **language model (LLM)**.  
Depending on the complexity of the task, chains can take different forms.

---

## 1. LLM Chain

**Concept:**  
The simplest type of chain that directly connects a prompt to an LLM.

**Function:**  
- Takes user input  
- Inserts it into the prompt  
- Sends it to the LLM  
- Returns the model’s response

**Use Case:**  
Used for simple, single-step tasks such as summarization or question answering.

---

## 2. Sequential Chain

**Concept:**  
A chain that executes multiple steps in sequence, where the output of one step becomes the input of the next.

**Function:**  
It operates like a pipeline in which each stage depends on the previous one.

**Use Case:**  
Used for multi-stage tasks, such as:  
1. Translating text  
2. Summarizing the translated version  
3. Generating a follow-up message

---

## 3. Router Chain

**Concept:**  
A smart chain that decides which subchain to send the input to, based on the content or type of the input.

**Function:**  
It dynamically selects the most appropriate subchain depending on the user’s input.

**Use Case:**  
When there are multiple types of questions or inputs, such as:  
- Math problems  
- History questions  
- Programming queries  
The router identifies the input type and routes it to the relevant chain.

---

## 4. Multi-Prompt Chain

**Concept:**  
A specialized version of the router chain. Instead of routing between entire chains, it routes between multiple prompt templates.

**Function:**  
Selects the most suitable prompt for a given input and then sends it to the language model.

**Use Case:**  
When using one model with different prompts for various domains, such as:  
- Math  
- History  
- Physics  
- Computer Science  
The multi-prompt chain determines which prompt best fits the input.

---

## Summary Table

| Chain Type | Description | Routes Between | Typical Use Case |
|-------------|--------------|----------------|------------------|
| **LLM Chain** | Connects a prompt directly to an LLM | — | Simple, single-step tasks |
| **Sequential Chain** | Connects multiple chains in sequence | Sequential steps | Multi-stage processes |
| **Router Chain** | Routes input to different subchains | Different chains | Handling different input types |
| **Multi-Prompt Chain** | Routes input to different prompts | Prompt templates | Selecting the right prompt for each question |


In [ ]:
!pip install  langchain langchain-core langchain-community langchain-openai python-dotenv


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence,RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key=openai_api_key,
)
model="meta-llama/llama-4-scout-17b-16e-instruct"


In [ ]:
llm = ChatOpenAI(
    model=model,
    temperature=0.9,
    api_key=openai_api_key,
)

# LLM Chain


In [ ]:
from langchain.chains import LLMChain




In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
product = "Deep Learning GPUs"
chain.run(product)

# Sequential Chains


In [ ]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    "Write a 30 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True)

In [ ]:
overall_simple_chain.run(product)


# Complex Sequential Chain

In [ ]:
from langchain.chains import LLMChain
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)


chain_one= LLMChain(llm=llm,prompt=first_prompt, output_key="English_Review")

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [ ]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [ ]:
from langchain.chains import SequentialChain
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
import pandas as pd
data_path =  f'{moneshsoni_customer_review_path}/Customer Review.csv'
df = pd.read_csv(data_path)
df.head()

In [ ]:
review = df.ReviewText[5]
overall_chain(review)

# 4. Router Chain


In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is Generative AI?")
